In [4]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from pyvis.network import Network as net

In [5]:
g = nx.read_gpickle("objects/graph-A_3_0")

In [6]:
print(nx.info(g))

DiGraph named 'Graph from soil food web adjacency matrix' with 24 nodes and 94 edges


In [7]:
NxG = net('1300px', '1300px')
# populates the nodes and edges data structures
NxG.from_nx(g)

In [15]:
# Import trophic level as node attribute
trophic_levels = pd.read_csv("data/trophic_levels_1_0.csv", index_col=0, header=None, delimiter=";", names=["trophic_level"])
trophic_levels = trophic_levels.squeeze()

# Create dict from trophic level Series
troph_lev_dict = trophic_levels.to_dict()

In [16]:
tlevel = list(troph_lev_dict.values())
# Change to fix palette and add manage problem if high number of torphic level (cyclical palette)!
colours = ["#40E0D0", "#DFFF00", "#FFBF00", "#DE3163"]
tlevel2color = {lev: colours[lev] for lev in tlevel}

def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))

# Create a PyVis graph G
G = net('800px', '1200px')
# nodes = list(g.nodes())
for node in g.nodes():
    G.add_node(node, 
               label='\n'.join(chunkstring(node,15)), 
               level=max(trophic_levels) - troph_lev_dict[node], 
               shape='box',
               color=tlevel2color[troph_lev_dict[node]])
    
# Add edges to PyVis graph
for source, target in list(g.edges()):
    G.add_edge(source, target)
    
# Edges take on the color of the node they are coming from.
G.inherit_edge_colors(status=True)

In [13]:
options= """
var options = {
  "edges": {
    "arrows": {
      "middle": {
        "enabled": true,
        "scaleFactor": 0.85
      }
    },
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "layout": {
    "hierarchical": {
      "enabled": true,
      "nodeSpacing": 200
    }
  },
  "interaction": {
    "hover": true,
    "navigationButtons": true,
    "multiselect": true
  },
  "manipulation": {
    "enabled": true,
    "initiallyActive": true
  },
  "physics": {
    "enabled": true,
    "hierarchicalRepulsion": {
      "centralGravity": 0,
      "nodeDistance": 190
    },
    "minVelocity": 0.75,
    "solver": "hierarchicalRepulsion"
  },
  "configure": {
    "enabled": true
  }
}
"""
# G.write_html('SFW_2.html')
G.conf = True
G.set_options(options)
# G.show_buttons(filter_=True)  # if we enable this after setting options there is a bug so we use conf=True and JS settings
G.write_html('SFW-1_0.html')